In [ ]:
import torch
import numpy as np
from preprocess import get_train_test_data
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from torchsummary import summary

train_data, train_labels, test_data = get_train_test_data()
train_data = torch.as_tensor(train_data, dtype=torch.float32)
test_data = torch.as_tensor(test_data, dtype=torch.float32)
train_labels = torch.as_tensor(train_labels, dtype=torch.int64)

#train_labels = F.one_hot(train_labels, num_classes=19)

In [ ]:
train_data = train_data.transpose(1, 2)
test_data = test_data.transpose(1, 2)
train_data.shape

In [ ]:
class LayerNorm1D(torch.nn.Module):     # pass
    def __init__(self, filters):
        super(LayerNorm1D, self).__init__()
        self.ln = torch.nn.LayerNorm(filters)

    def forward(self, x):
        x = x.transpose(1, 2)
        x = self.ln(x)
        return x.transpose(1, 2)

In [ ]:
class res_block(torch.nn.Module):
    def __init__(self, C, L, filters, kernal_size):
        super(res_block, self).__init__()

        self.conv1d = torch.nn.Sequential(
            torch.nn.Conv1d(C, filters, 1),
            torch.nn.ReLU(), LayerNorm1D(filters),

            torch.nn.Conv1d(filters, filters, kernal_size, padding=kernal_size//2),
            torch.nn.ReLU(), LayerNorm1D(filters),

            torch.nn.Conv1d(filters, filters, 1),
            torch.nn.ReLU(), LayerNorm1D(filters),
        )

        self.shortcut = torch.nn.Conv1d(C, filters, 1)
    
    def forward(self, x):
        return self.conv1d(x) + self.shortcut(x)

class inception_block(torch.nn.Module):
    def __init__(self, filters=128, kernal_size=5):
        super(inception_block, self).__init__()

        self.output_shape = filters // 2

        self.block = torch.nn.Sequential(
            res_block(8, 61, filters, kernal_size),
            torch.nn.MaxPool1d(2, 2),       # pass
            torch.nn.Dropout2d(0.3),        # pass
            res_block(128, 30, filters//2, kernal_size),
            torch.nn.AdaptiveAvgPool1d(1),  # pass
        )

    def forward(self, x):
        x = self.block(x)
        return x.squeeze(-1)

In [ ]:
class ComplexConv1D(torch.nn.Module):
    def __init__(self):
        super(ComplexConv1D, self).__init__()

        self.seq_3 = inception_block(kernal_size=3)
        self.seq_5 = inception_block(kernal_size=5)
        self.seq_7 = inception_block(kernal_size=7)
        
        dense_in_size = self.seq_3.output_shape + self.seq_5.output_shape + self.seq_7.output_shape

        self.dense = torch.nn.Sequential(
            torch.nn.Linear(dense_in_size, 512), torch.nn.ReLU(), torch.nn.Dropout(0.3),
            torch.nn.Linear(512, 128), torch.nn.ReLU(), torch.nn.Dropout(0.3),
            torch.nn.Linear(128, 19)
        )

    def forward(self, x):
        x = torch.cat([self.seq_3(x), self.seq_5(x), self.seq_7(x)], axis=-1)
        return self.dense(x)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(train_data, train_labels,
                        test_size=0.2, random_state=7)
x_train.shape, x_val.shape, y_train.shape, y_val.shape

In [ ]:
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss
from ignite.contrib.handlers.param_scheduler import LRScheduler

train_loader = DataLoader(TensorDataset(x_train, y_train), batch_size=64, shuffle=True)
val_loader = DataLoader(TensorDataset(x_val, y_val), batch_size=64, shuffle=False)
#test_loader = DataLoader(TensorDataset(test_data, None), batch_size=64, shuffle=False)

def scheduler(epoch):
    if epoch < 30:
        rate = 1.0
    elif epoch >= 30 and epoch < 45:
        rate = 0.3
    elif epoch >= 45:
        rate = 0.3**2
    return rate

def ce_loss(output, target):
    #output = (1.0 - 1e-6) * output + 1e-7
    eps = 0.1
    c = output.shape[-1]
    target = F.one_hot(target, c) * (1 - eps) + eps / c
    log_preds = F.log_softmax(output, dim=-1)
    return -(target * log_preds).sum(dim=-1).mean()

model = ComplexConv1D().to(device='cuda')

def weights_init(m):
    if isinstance(m, torch.nn.Linear) or isinstance(m, torch.nn.Conv1d):
        torch.nn.init.xavier_uniform_(m.weight.data)
        torch.nn.init.zeros_(m.bias.data)
model.apply(weights_init)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = ce_loss

lr_scheduler = LRScheduler(
    torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=scheduler)
)

trainer = create_supervised_trainer(model, optimizer, loss_fn, device='cuda')

evaluator = create_supervised_evaluator(model,
        {'acc': Accuracy(), 'loss': Loss(loss_fn)}, device='cuda')

# summary(model, input_size=(8, 61))

In [ ]:
@trainer.on(Events.EPOCH_COMPLETED)
def on_epoch_end():
    str1 = log_training_results(trainer)
    str2 = log_validation_results(trainer)
    print(str1 + ' - ' + str2 + ' - lr: ' + '%.1e' % optimizer.param_groups[0]['lr'])
    lr_scheduler(trainer)

def log_training_results(trainer):
    evaluator.run(train_loader)
    metrics = evaluator.state.metrics
    return ("Epoch {}/{} - loss: {:.4f} - acc: {:.4f}"
        .format(trainer.state.epoch, trainer.state.max_epochs, metrics["loss"], metrics["acc"]))

def log_validation_results(trainer):
    evaluator.run(val_loader)
    metrics = evaluator.state.metrics
    return ("val_loss: {:.4f} - val_acc: {:.4f}".format(metrics["loss"], metrics["acc"]))

import time

start = time.time()

trainer.run(train_loader, max_epochs=55)

time.time() - start